# Data Cleaning 


**CS 109a: Airbnb**

**Richa Chaturvedi, Mirai Shah, and Sam Plank**

In [1]:
import numpy as np
from scipy import sparse
import re
import matplotlib
import pandas as pd
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.cm as cmx
import matplotlib.colors as colors
import datetime as dt
import math
from sklearn.linear_model import Lasso as Lasso_Reg
import numpy as np 
import sklearn.preprocessing as Preprocessing
from sklearn.preprocessing import StandardScaler as Standardize
%matplotlib inline
from sklearn.linear_model import LinearRegression as Lin_Reg
from sklearn.linear_model import Ridge as Ridge_Reg

### Clean Calendar Dataset

The calendar dataset gives the price of listings over the course of a year.  In order to clean this dataset, we first renamed the columns with more appropriate labels.  We then stripped the dates and prices of their extraneous characters so they were easier to work with.  Finally, we converted the column types to apropriate dytpes and filtered out columns with no information.

In [ ]:
# open calendar csv file
calendar = pd.read_csv('calendar.csv', delimiter = '\t') 

# rename columns 
calendar["listing"], calendar['date'], calendar['available'], calendar['price'] = zip(*calendar['listing_id,"date","available","price"'].str.split(',').tolist())
calendar = calendar.drop(['listing_id,"date","available","price"'], 1)

# change column contents to be more workable format
calendar['date'] = calendar['date'].map(lambda x: x.lstrip('"').rstrip('"'))
calendar['price'] = calendar['price'].map(lambda x: x.lstrip('$').rstrip('.'))

# change column dtypes 
calendar['price'] = pd.to_numeric(calendar['price'])
calendar['date'] = pd.to_datetime(calendar['date'])
calendar['listing'] = pd.to_numeric(calendar['listing'])

# filter out the columns with no prices 
calendar=calendar[calendar['available'] == 't']

### Clean Listings Dataset

The listings dataset is our main dataset for this project.  It contains thousands of rows of listing information for Airbnbs in New York City.  We attempted to clean the data many different ways before settling on our final dataset.  One notable technique we tried was KNN.  Ultimately, this was unsuccessful because most rows had some NaN values and it was therefore difficult to find similar listings.  KNN ended up being too costly and ineffective.

First, we dropped the columns with null values that added no values to the dataset.  We then dropped null values that could not be salvaged (things that could not be filled in based off of any known technique or our intuition such as ID or property type).  

Next, we changed the format of two variables (price and extra people) to integers rather than objects so we could easily perform statistical procedures with the information provided.  The zipcodes were given in their extended form, so we decided to only use the first five numbers.  

For missing weekly and monthly prices, we used scaled nightly price to fill in the missing data.  Finally, we replaced some (host since, etc.) with the columns from our sentiment analysis dataset.

In [ ]:
# open dataset with information and pricing for each listing 
df = pd.read_csv('listings.csv',encoding='latin1')

In [ ]:
# drop columns with null values that don't add important information to dataset
df = df.drop(['country', 'neighbourhood', 'square_feet', 'state'], 1)

# drop null values that can't be salvaged 
df = df[(pd.notnull(df['id']))&(pd.notnull(df['host_id']))&(pd.notnull(df['zipcode']))&(pd.notnull(df['latitude']))]
df = df[(pd.notnull(df['longitude']))&(pd.notnull(df['bathrooms']))&(pd.notnull(df['bedrooms']))&(pd.notnull(df['beds']))]
df = df[(pd.notnull(df['property_type']))&(pd.notnull(df['price']))&((df.number_of_reviews!=0)&(pd.notnull(df.review_scores_rating)))]

# reset index after dropping certain rows
df = df.reset_index(drop=True)

# change format of prices and extra_people to integer rather than object
prices = []
extra_people = []
for i in range(len(df)):
    price = int(float(str(df['price'][i]).replace('$', '').replace(',', '')))
    extra_person = int(float(str(df['extra_people'][i]).replace('$', '').replace(',', '')))
    prices.append(price)
    extra_people.append(extra_person)

df['price'] = prices
df['extra_people'] = extra_people

# only keep first five numbers of zipcode 
zipcodes = []
for i in range(len(df)):
    zipcode = df['zipcode'][i][:5]
    zipcodes.append(zipcode)
df['zipcode'] = zipcodes

# fill the null values in weekly_prices and monthly_prices column
# change the dtype from object to integer in these two columns
wprices = []
mprices = []

for i in range(len(df)):
    if pd.isnull(df['weekly_price'][i]):
        wprice = df['price'][i] * 7
    elif pd.notnull(df['weekly_price'][i]):
        wprice = int(float(str(df['weekly_price'][i]).replace('$', '').replace(',', '')))
    wprices.append(wprice)
    if pd.isnull(df['monthly_price'][i]):
        mprice = df['price'][i] * 30
    elif pd.notnull(df['monthly_price'][i]):
        mprice = int(float(str(df['monthly_price'][i]).replace('$', '').replace(',', '')))
    mprices.append(mprice)
    
df['weekly_price'] = wprices
df['monthly_price'] = mprices

df = df.dropna(axis=0)
df = df.reset_index(drop=True)

In [ ]:
# drop the original host_since, first_review, last_review
df = df.drop(['host_since', 'first_review', 'last_review'], 1)

# add in new csv with the sentiment columns and converted host_since, first_review, last_review columns
sentiment = pd.read_csv('time_sentiment.csv')

host_since = sentiment['host_since'] 
first_review = sentiment['first_review'] 
last_review = sentiment['last_review'] 
sentiments = sentiment['sentiment']

frames = [host_since, first_review, last_review, sentiments]
final_sentiment = pd.concat(frames, 1)
final_sentiment = final_sentiment.dropna(axis=0, how='any')
frames_new = [df, final_sentiment]

df_final = pd.concat(frames_new, 1)